In [1]:
import cv2
from imutils.video import VideoStream

In [2]:
dnn_proto_text='./models/face_detection/deploy.prototxt'
dnn_model='./models/face_detection/res10_300x300_ssd_iter_140000.caffemodel'
face_ssd = cv2.dnn.readNetFromCaffe(dnn_proto_text, dnn_model)


windows_name =  "Detect Face"
video = VideoStream(1 + cv2.CAP_DSHOW).start()


In [5]:
threshold = 0.70

while True: 
    image = video.read()
    
    H, W, _ = image.shape

    # cv2.dnn.blobFromImage()
    face_ssd.setInput(cv2.dnn.blobFromImage(
        image, 1.0, (150, 150), (104.0, 177.0, 123.0), False, False))
    detections = face_ssd.forward()

    for result in detections[0, 0, :, :]:
        confidence = result[2]
        if confidence > threshold:
            x_left_bottom = int(result[3] * W)
            y_left_bottom = int(result[4] * H)
            x_right_top = int(result[5] * W)
            y_right_top = int(result[6] * H)
            
            facebox = [x_left_bottom, y_left_bottom, x_right_top, y_right_top ]

            cv2.rectangle(image, (facebox[0], facebox[1]),
                      (facebox[2], facebox[3]), (0, 255, 0))
            label = "face: %.2f" % confidence
            label_size, base_line = cv2.getTextSize(
                label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

            cv2.rectangle(image, (facebox[0], facebox[1] - label_size[1]),
                      (facebox[0] + label_size[0],
                       facebox[1] + base_line),
                      (0, 255, 0), cv2.FILLED)
            cv2.putText(image, label, (facebox[0], facebox[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
            
            
    cv2.imshow(windows_name, image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cv2.destroyAllWindows()